In [49]:
import urllib.request
import re
import pandas as pd
import datetime
from pandas.tseries.offsets import BDay

from pandas.io import data, wb
import os.path
import math
import numpy

import talib as ta

In [50]:
def get_support_resistance(symbol):
    url = 'http://www.stockta.com/cgi-bin/analysis.pl?symb={0}&cobrand=&mode=stock'.format(symbol)
    
    support = (0, 0)
    resistance = (0, 0)
    with urllib.request.urlopen(url) as f:
        contend = f.read().decode('iso-8859-1')
       
        match = re.search(r'&support=(.*)&', contend)
        if match:
            levels_str = re.findall(r'([\d.]+),([\d.]+)', match.group())
            levels_num = [(float(t[0]), int(t[1])) for t in levels_str]
            supports = [x for x in levels_num if x[0] < price and x[1] >= 5 ]
            if supports:                
                support =  sorted(supports).pop()
            resistances = [x for x in levels_num if x[0] > price and x[1] >=5]
            if resistances:
                resistance = sorted(resistances)[0]
            
    return support, resistance

In [51]:
def update_ohlc(symbol):
    
    today = datetime.datetime.today()
    
    #update ohlc data
    if (not os.path.isfile('data/'+symbol+'-ohlc.csv')):
        df_train = data.DataReader(symbol, 'yahoo', datetime.datetime(1900, 2, 1), today)

    else:
        df_train = pd.read_csv('data/'+symbol+'-ohlc.csv', index_col='Date', parse_dates=True)
        start = df_train.index[-1] + datetime.timedelta(days=1)
        print(start,today)

        if(start <= today):
            new_data = data.DataReader(symbol, 'yahoo', start, today )
            print(new_data)
            df_train = df_train.append(new_data)

    df_train.to_csv('data/'+symbol+'-ohlc.csv')
    
    #update dividends info
    filename_dividends = 'data/'+symbol+'-dividends.csv'

    
    url_temp = 'http://ichart.yahoo.com/table.csv?s={s}&c={c}&a={a}&b={b}&f={f}&d={d}&e={e}&g=v&ignore=.csv'
    #if (not os.path.isfile(filename_dividends)): #create new dividend file
    url = url_temp.format(s=symbol, a='1', b='1', c='1980',d=today.month, e=today.day, f=today.year)
    print(url)
    df_dividends = pd.read_csv(url, index_col=0, parse_dates=[0])
    if len(df_dividends) > 0:        
        #sort to ascending order, match that of ohlc
        df_dividends.sort_index(inplace=True)
        df_dividends.to_csv(filename_dividends) 

In [52]:
def get_more_info(sym, atr_period=10):
    
    update_ohlc(sym)
    df_train = pd.read_csv('data/'+sym+'-ohlc.csv', index_col='Date', parse_dates=True)
    if (os.path.isfile('data/'+sym+'-dividends.csv')):
        df_dividends = pd.read_csv('data/'+sym+'-dividends.csv', index_col='Date', parse_dates=True)
        df_train = df_train.join(df_dividends)
        df_train['Dividends'].fillna(0, inplace=True)
    else:
        df_train['Dividends']=0
        
    high = df_train['High'].values
    low = df_train['Low'].values
    close = df_train['Close'].values
    
    atr = round_2d(ta.ATR(high, low, close, atr_period)[-1])
    prev_close = round_2d(close[-1])
    print(prev_close, atr)
    return prev_close, atr

In [53]:
def round_2d(price):
    return math.ceil(price*100)/100

In [54]:
f = open('sp500-tickers-test.txt', 'r')
symbols = f.read().split()

next_bday = datetime.date.today()  + BDay(1)
next_bday_str = next_bday.strftime('%Y-%m-%d')

index = 0
level_max_distance = 1.1

df_levels = pd.DataFrame(columns=['Date','symbol','support', 'sup_li','resistance', 'res_li','prev_day_close','atr'])
for sym in symbols:
    try:
        support, resistance = get_support_resistance(sym)
        print(support, resistance)
        #output only if support or resistance exist and less than 5 percent away from current price
        if (support[0] >0 and support[1] >= 10 ) or (resistance[0] >0 and resistance[1] >= 10): 
            prev_close, atr = get_more_info(sym)
            if prev_close/support[0] < level_max_distance or resistance[0]/prev_close < level_max_distance:
                df_levels.loc[index] = [next_bday_str, sym, support[0], support[1], resistance[0], resistance[1], prev_close, atr]
                index = index+1
    except Exception as e:
        print(str(e))

df_levels.to_csv('sup-res-levels.txt')
df_levels.to_csv('sup-res-levels-'+next_bday_str+'.txt')
print("processing completed")



(110.84, 13) (114.32, 6)
2016-04-30 00:00:00 2016-05-02 12:20:05.455145
Empty DataFrame
Columns: [Open, High, Low, Close, Volume, Adj Close]
Index: []
http://ichart.yahoo.com/table.csv?s=AET&c=1980&a=1&b=1&f=2016&d=5&e=2&g=v&ignore=.csv
112.27 2.14
processing completed
